In [ ]:
import matplotlib.pyplot as plt
import os
import json
import cv2
from collections import defaultdict
import numpy as np
from hico_text import hico_text_label
​
hico2coco = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                        24, 25, 27, 28, 31, 32, 33, 34, 35, 36,
                        37, 38, 39, 40, 41, 42, 43, 44, 46, 47,
                        48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
                        58, 59, 60, 61, 62, 63, 64, 65, 67, 70,
                        72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                        82, 84, 85, 86, 87, 88, 89, 90)
coco2hico = {v:i for i, v in enumerate(hico2coco)}
​
hoi_anno_path = '/mnt/HDD/hico_20160224_det/hico_list_hoi.txt'
verb_anno_path = '/mnt/HDD/hico_20160224_det/hico_list_vb.txt'
# obj_anno_path = '/mnt/HDD/hico_20160224_det/hico_list_obj.txt'
​
with open(hoi_anno_path, 'r') as f:
    hoi_anno = f.readlines()
with open(verb_anno_path, 'r') as f:
    verb_anno = f.readlines()
# with open(obj_anno_path, 'r') as f:
#     obj_anno = f.readlines()
​
hoi_anno = [h.strip().split() for h in hoi_anno[2:]]
verb_anno = [v.strip().split() for v in verb_anno[2:]]
# obj_anno = [o.strip().split() for o in obj_anno[2:]]
​
verb_idx2class = {int(v[0]) : v[1] for v in verb_anno}
obj_idx2class = {1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 
                 11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 
                 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 
                 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 
                 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 
                 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 
                 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table', 
                 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven', 
                 80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock', 86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 
                 90: 'toothbrush'}
for k,v in obj_idx2class.items():
    if len(v.split()) > 1:
        obj_idx2class[k] = '_'.join(v.split())
​
verb_class2idx = {v:k for k,v in verb_idx2class.items()}
obj_class2idx = {v:k for k,v in obj_idx2class.items()}
​
obj_in_verb = defaultdict(list)
​
for h in hoi_anno:
    idx, obj, verb = h
    verb_idx = verb_class2idx[verb]
    obj_idx = obj_class2idx[obj]
    obj_in_verb[verb_idx].append(obj_idx)
    
anno_file = '/mnt/HDD/hico_20160224_det/annotations/trainval_hico.json'
# anno_file = '/mnt/HDD/hico_20160224_det/annotations/trainval_hico_diffusion_replace.json'
# anno_file = '/mnt/HDD/hico_20160224_det/annotations/test_hico.json'
​
with open(anno_file, 'r') as f:
    annotations = json.load(f)
    
root_path = '/mnt/HDD/hico_20160224_det/images/train2015'
# root_path = '/mnt/HDD/hico_20160224_det/images/test2015'
idx = 28
img_path = annotations[idx]['file_name']
img = cv2.imread(os.path.join(root_path,img_path))
ori_img = img.copy()
print(img_path)

results = []
for k, anno in enumerate(annotations[idx]['hoi_annotation']):
    sub_id = anno['subject_id']
    obj_id = anno['object_id']
    sub_info = annotations[idx]['annotations'][sub_id]
    obj_info = annotations[idx]['annotations'][obj_id]
    
    sub_category_id = sub_info['category_id']
    obj_category_id = obj_info['category_id']
    
    sub_category = obj_idx2class[sub_category_id]
    obj_category = obj_idx2class[obj_category_id]
    
    sub_bbox = sub_info['bbox']
    obj_bbox = obj_info['bbox']
    
    sub_center = ((sub_bbox[0] + sub_bbox[2])//2, (sub_bbox[1] + sub_bbox[3])//2)
    obj_center = ((obj_bbox[0] + obj_bbox[2])//2, (obj_bbox[1] + obj_bbox[3])//2)
    
    verb_category = verb_idx2class[anno['category_id']]
    # print(anno['hoi_category_id'])
    
    cv2.rectangle(img, sub_bbox[:2], sub_bbox[2:], (0,255,0), 3)
    cv2.rectangle(img, obj_bbox[:2], obj_bbox[2:], (255,0,0), 3)
    cv2.line(img, sub_center, obj_center, (0,0,255), thickness=3)
    
    verb_loc = [(sub_center[0] + obj_center[0])//2, (sub_center[1] + obj_center[1])//2]
    verb_loc[1] += k*20
    cv2.putText(img, sub_category, sub_bbox[:2], cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    cv2.putText(img, obj_category, obj_bbox[:2], cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    cv2.putText(img, verb_category, verb_loc, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
    
    result_dict = {}
    result_dict['verb'] = verb_category
    result_dict['object'] = obj_category
    result_dict['sub_bbox'] = sub_bbox
    result_dict['obj_bbox'] = obj_bbox
    
    results.append(result_dict)
    
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
axes[0].imshow(ori_img[...,::-1])
axes[0].set_title('Image')
axes[0].axis('off')
​
axes[1].imshow(img[...,::-1])
axes[1].set_title('GT')
axes[1].axis('off')
​
# plt.show()
​
verb_idx=verb_class2idx[results[0]['verb']]
print(results[0]['verb'])
print([obj_idx2class[idx] for idx in obj_in_verb[verb_idx]])
print([hico_text_label[(verb_idx-1, coco2hico[idx])] for idx in obj_in_verb[verb_idx]])
​
data = 'train'
​
if data == 'train':
​
    # anno_file = '/mnt/HDD/hico_20160224_det/annotations/trainval_hico.json'
    anno_file = '/mnt/HDD/hico_20160224_det/annotations/trainval_hico_diffusion_replace.json'
​
    with open(anno_file, 'r') as f:
        annotations = json.load(f)
​
    annotations = annotations[37633:]
    
    num_of_obj = defaultdict(int)
    num_of_verb = defaultdict(int)
    num_of_hoi = defaultdict(int)
    hoi2verbobj = {}
    max_verb = 0
    for idx, anno in enumerate(annotations):
        for ins in anno['annotations']:
            obj_category_id = ins['category_id']
            num_of_obj[obj_category_id] += 1
​
        for ins in anno['hoi_annotation']:
            verb_category_id = ins['category_id'] # 1~117
            hoi_category_id = ins['hoi_category_id'] # 1~600
            
            if hoi_category_id == 26:
                print('this')
                pass
            if hoi_category_id == 130:
                print('this')
                pass
            num_of_verb[verb_category_id] += 1
            num_of_hoi[hoi_category_id] += 1
                
            sub_id = ins['subject_id']
            obj_id = ins['object_id']
            
            hoi2verbobj[hoi_category_id] = (verb_category_id, anno['annotations'][obj_id]['category_id'])
else:        
    anno_file = '/mnt/HDD/hico_20160224_det/annotations/test_hico.json'
​
    with open(anno_file, 'r') as f:
        annotations = json.load(f)
​
    num_of_obj = defaultdict(int)
    num_of_verb = defaultdict(int)
    num_of_hoi = defaultdict(int)
    hoi2verbobj = {}
​
    hico_key2idx = {k:i for i, k in enumerate(list(hico_text_label.keys()))}
​
    for anno in annotations:
        for ins in anno['annotations']:
            obj_category_id = ins['category_id']
            num_of_obj[obj_category_id] += 1
​
        for ins in anno['hoi_annotation']:
            verb_category_id = ins['category_id'] # 1~117
            num_of_verb[verb_category_id] += 1            
            obj_id = ins['object_id']
            obj_cat_id = anno['annotations'][obj_id]['category_id']
            
            hoi_category_id = hico_key2idx[(verb_category_id-1, coco2hico[obj_cat_id])]
​
            num_of_hoi[hoi_category_id] += 1
​
            hoi2verbobj[hoi_category_id] = (verb_category_id, obj_cat_id)
            
​
new_obj_dict = {}
for k,v in num_of_obj.items():
    new_obj_dict[obj_idx2class[k]] = v
​
new_verb_dict = {}
for k,v in num_of_verb.items():
    new_verb_dict[verb_idx2class[k]] = v
​
new_hoi_dict = {}
aa = []
for k,v in num_of_hoi.items():
    verb_category_id, obj_category_id = hoi2verbobj[k]
    hoi_name = verb_idx2class[verb_category_id] + '_' + obj_idx2class[obj_category_id]
    aa.append(hoi_name)
    new_hoi_dict[hoi_name] = v
    
sorted_obj_dict = sorted(new_obj_dict.items(), key = lambda item: item[1], reverse = True)
sorted_verb_dict = sorted(new_verb_dict.items(), key = lambda item: item[1], reverse = True)
sorted_hoi_dict = sorted(new_hoi_dict.items(), key = lambda item: item[1], reverse = True)

: 

In [ ]:
# Explore the number of entries in the dataset
print("Number of entries in the dataset:", len(hico_dataset))

# Access a specific entry
entry = hico_dataset[0]  # Get the first entry, for example

# Print out details of the entry to understand its structure
print(json.dumps(entry, indent=2))
